In [1]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

In [2]:
# Load the data
final_merged_df = pd.read_csv('final_merged_df.csv')
# Define target and features
y = final_merged_df[['co2_avg', 'pm10_avg', 'pm25_avg', 'no2_avg']]
X = final_merged_df.drop(columns=['co2_avg', 'pm10_avg', 'pm25_avg', 'no2_avg'])

# Drop aadt columns (keeping vkm columns)
X.drop(columns=['aadt_motorcycle', 'aadt_taxi',
                'aadt_petrol_car', 'aadt_diesel_car', 'aadt_electric_car',
                'aadt_petrol_phv', 'aadt_diesel_phv', 'aadt_electric_phv',
                'aadt_petrol_lgv', 'aadt_diesel_lgv', 'aadt_electric_lgv',
                'aadt_hgvs_rigid_2_axles', 'aadt_hgvs_rigid_3_axles',
                'aadt_hgvs_rigid_4_or_more_axles', 'aadt_hgvs_articulated_3_to_4_axles',
                'aadt_hgvs_articulated_5_axles', 'aadt_hgvs_articulated_6_axles',
                'aadt_buses', 'aadt_coaches'], inplace=True)

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Hyperparameters tuning

In [30]:
# parameter grid for the MLPRegressor.
param_grid = {
    'regressor__mlp__hidden_layer_sizes': [(64, 128), (128, 256), (128, 256, 128)],
    'regressor__mlp__activation': ['relu', 'tanh'],
    'regressor__mlp__solver': ['adam', 'lbfgs'],
    'regressor__mlp__learning_rate_init': [0.001, 0.01]
}

In [31]:
# Initialize the MLPRegressor.
mlp = MLPRegressor(max_iter=2000, random_state=42)

In [32]:
# Set up K-fold cross validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Define two sets of scalers:
scalers = {
    'StandardScaler': (StandardScaler(), StandardScaler()),
    'MinMaxScaler': (MinMaxScaler(), MinMaxScaler())
}

In [33]:
results = {}

# Loop over the scaling options
for scaler_name, (x_scaler, y_scaler) in scalers.items():
    print(f"Running grid search with {scaler_name} for both features and target...")
    
    # Create a pipeline for the features:
    # First, scale the features, then apply the MLPRegressor.
    pipeline = Pipeline([
        ('scaler', x_scaler),
        ('mlp', MLPRegressor(max_iter=500, random_state=42))
    ])
    
    # Wrap the pipeline with TransformedTargetRegressor to scale the target values.
    # This ensures that during prediction, the outputs are automatically inverse transformed
    # back to the original (unnormalized) space.
    tt_reg = TransformedTargetRegressor(regressor=pipeline, transformer=y_scaler)
    
    # Set up GridSearchCV for the current pipeline
    grid_search = GridSearchCV(estimator=tt_reg,
                               param_grid=param_grid,
                               cv=kfold,
                               scoring='neg_mean_squared_error',
                               n_jobs=-1,
                               verbose=3)
    
    # Fit the grid search on the training data
    grid_search.fit(X_train, y_train)
    
    # Predict on the test set; note that predictions are automatically inverse-transformed.
    y_pred = grid_search.predict(X_test)
    
    # Compute evaluation metrics on the unscaled values
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results[scaler_name] = {
        'grid_search': grid_search,
        'mse': mse,
        'mae': mae,
        'r2': r2,
        'best_params': grid_search.best_params_
    }
    
    print(f"Results for {scaler_name}:")
    print(f"  Best Parameters: {grid_search.best_params_}")
    print(f"  Test MSE: {mse:.4f}")
    print(f"  Test MAE: {mae:.4f}")
    print(f"  Test R²: {r2:.4f}\n")

Running grid search with StandardScaler for both features and target...
Fitting 5 folds for each of 24 candidates, totalling 120 fits


/Users/minhanh29/anaconda3/envs/tts/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/minhanh29/anaconda3/envs/tts/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/minhanh29/anaconda3/envs/tts/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWa

[CV 2/5] END regressor__mlp__activation=relu, regressor__mlp__hidden_layer_sizes=(64, 128), regressor__mlp__learning_rate_init=0.01, regressor__mlp__solver=adam;, score=-4.467 total time=   3.7s
[CV 1/5] END regressor__mlp__activation=relu, regressor__mlp__hidden_layer_sizes=(64, 128), regressor__mlp__learning_rate_init=0.01, regressor__mlp__solver=lbfgs;, score=-3.981 total time= 1.3min


/Users/minhanh29/anaconda3/envs/tts/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/minhanh29/anaconda3/envs/tts/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV 5/5] END regressor__mlp__activation=relu, regressor__mlp__hidden_layer_sizes=(64, 128), regressor__mlp__learning_rate_init=0.001, regressor__mlp__solver=adam;, score=-1.762 total time=   3.7s
[CV 5/5] END regressor__mlp__activation=relu, regressor__mlp__hidden_layer_sizes=(64, 128), regressor__mlp__learning_rate_init=0.01, regressor__mlp__solver=adam;, score=-5.875 total time=   3.6s
[CV 3/5] END regressor__mlp__activation=relu, regressor__mlp__hidden_layer_sizes=(64, 128), regressor__mlp__learning_rate_init=0.01, regressor__mlp__solver=lbfgs;, score=-2.668 total time= 1.3min


...

Results for StandardScaler:
  Best Parameters: {'regressor__mlp__activation': 'relu', 'regressor__mlp__hidden_layer_sizes': (128, 256, 128), 'regressor__mlp__learning_rate_init': 0.001, 'regressor__mlp__solver': 'lbfgs'}
  Test MSE: 5.6398
  Test MAE: 0.2402
  Test R²: 0.9534

Running grid search with MinMaxScaler for both features and target...
Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 2/5] END regressor__mlp__activation=relu, regressor__mlp__hidden_layer_sizes=(64, 128), regressor__mlp__learning_rate_init=0.001, regressor__mlp__solver=adam;, score=-5.729 total time=   2.2s
[CV 2/5] END regressor__mlp__activation=relu, regressor__mlp__hidden_layer_sizes=(64, 128), regressor__mlp__learning_rate_init=0.01, regressor__mlp__solver=lbfgs;, score=-11.279 total time=  13.4s


/Users/minhanh29/anaconda3/envs/tts/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV 4/5] END regressor__mlp__activation=relu, regressor__mlp__hidden_layer_sizes=(64, 128), regressor__mlp__learning_rate_init=0.001, regressor__mlp__solver=adam;, score=-3.415 total time=   2.2s
[CV 1/5] END regressor__mlp__activation=relu, regressor__mlp__hidden_layer_sizes=(64, 128), regressor__mlp__learning_rate_init=0.01, regressor__mlp__solver=lbfgs;, score=-12.851 total time=  14.9s
[CV 2/5] END regressor__mlp__activation=relu, regressor__mlp__hidden_layer_sizes=(64, 128), regressor__mlp__learning_rate_init=0.01, regressor__mlp__solver=adam;, score=-17.372 total time=   2.1s
[CV 3/5] END regressor__mlp__activation=relu, regressor__mlp__hidden_layer_sizes=(64, 128), regressor__mlp__learning_rate_init=0.01, regressor__mlp__solver=lbfgs;, score=-11.229 total time=  16.0s
[CV 3/5] END regressor__mlp__activation=relu, regressor__mlp__hidden_layer_sizes=(64, 128), regressor__mlp__learning_rate_init=0.01, regressor__mlp__solver=adam;, score=-5.617 total time=   2.1s
[CV 4/5] END regres

/Users/minhanh29/anaconda3/envs/tts/lib/python3.10/site-packages/sklearn/model_selection/_search.py:409: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.array(param_list)


Results for MinMaxScaler:
  Best Parameters: {'regressor__mlp__activation': 'relu', 'regressor__mlp__hidden_layer_sizes': (64, 128), 'regressor__mlp__learning_rate_init': 0.001, 'regressor__mlp__solver': 'adam'}
  Test MSE: 6.5689
  Test MAE: 0.8989
  Test R²: 0.9431



In [34]:
# Save the best MLP model
joblib.dump(grid_search.best_estimator_, './checkpoints/mlp_model.pkl')

['mlp_model.pkl']

#### Train best params model on full training data

In [4]:
x_scaler = StandardScaler()
y_scaler = StandardScaler()

x_scaler.fit(X_train)
y_scaler.fit(y_train)
X_train_transformed = x_scaler.transform(X_train)
X_test_transformed = x_scaler.transform(X_test)
y_train_transformed = y_scaler.transform(y_train)

In [5]:
X_train_transformed

array([[-0.88700646, -0.03041644, -0.23173864, ..., -0.64636844,
         2.32774421, -1.01283628],
       [ 0.33417649, -0.28979127, -0.19283918, ..., -0.64636844,
         2.32774421, -1.01283628],
       [ 0.33417649, -0.21339126, -0.22067937, ...,  1.54710523,
        -0.42960047, -1.01283628],
       ...,
       [-0.88700646, -0.14978573, -0.18876875, ..., -0.64636844,
        -0.42960047, -1.01283628],
       [-0.88700646, -0.15826952, -0.23853549, ..., -0.64636844,
         2.32774421, -1.01283628],
       [-0.88700646, -0.30295261, -0.24210671, ..., -0.64636844,
        -0.42960047,  0.98732641]])

In [6]:
y_train_transformed

array([[ 0.08457314,  0.31747617,  0.26980409,  0.11636966],
       [-0.1378209 , -0.16049027, -0.1577441 , -0.14071574],
       [-0.15949035, -0.24089412, -0.22688975, -0.17272671],
       ...,
       [-0.18451346, -0.30781913, -0.28519817, -0.20371039],
       [-0.07854069, -0.01295726, -0.02466216, -0.086304  ],
       [-0.19031977, -0.29341884, -0.27329912, -0.21269237]])

In [7]:
# best params
mlp = MLPRegressor(
    max_iter=2000, 
    random_state=42,
    activation='relu',
    hidden_layer_sizes=(128, 256, 128),
    learning_rate_init=0.001,
    solver='lbfgs'
)

In [ ]:
mlp.fit(X_train_transformed, y_train_transformed)

#### Evaluation

In [ ]:
y_pred = mlp.predict(X_test_transformed)

# transform back to unstandardized values
y_pred = y_scaler.inverse_transform(y_pred)

In [ ]:
# Compute evaluation metrics on the unscaled values
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"  Test MSE: {mse:.4f}")
print(f"  Test MAE: {mae:.4f}")
print(f"  Test R²: {r2:.4f}\n")

In [ ]:
# Save the best MLP model
joblib.dump(mlp, './checkpoints/mlp_model_full_training_data.pkl')
joblib.dump(x_scaler, './checkpoints/mlp_model_x_scaler.pkl')
joblib.dump(y_scaler, './checkpoints/mlp_model_y_scaler.pkl')